In [1]:
import os
import re
import spacy
import pandas as pd
from

# Preprocess Text

In [4]:
FILE_PATH = "../assets/NER_assets/Ancient_Words.csv"
# read csv file
df = pd.read_csv(FILE_PATH)
# Preprocess Text

In [5]:
# rename columns to fit code
df.rename(columns = {'Word':'Keyword', 'Category Types':'Label'}, inplace = True)
# If a cell is empty (NaN), Fill it with the value in its parallel "Early" column
for row in df:
    df['Quote'].fillna(df['Early Quote'], inplace=True)
    df['Word Before'].fillna(df['Early Word Before'], inplace=True)
    df['Word After'].fillna(df['Early Word After'], inplace=True)
    df['Label'].fillna(df['Early Category Type'], inplace=True)
# remove rows with no Keyword
df = df.dropna(subset=['Keyword'])
# Remove any row that isn't Greek
pat = '[ء-ي]+'
#df.Keyword.str.contains(pat)
df = df[~df.Keyword.str.contains(pat, na=False)]
#replace new line in df column
df['Keyword'].replace('\n', '', regex=True, inplace=True)
#replace numbers in df
df.replace('\d+', '', regex=True, inplace=True)
#replace hyphens in df column
df.replace('-', '', regex=True, inplace=True)
# replace comma in df column
df['Keyword'].replace(',', '', regex=True, inplace=True)
#replace period in df column
df['Keyword'].replace('\.', '', regex=True, inplace=True)
#replace interpunkt in df column
df['Keyword'].replace('\·', '', regex=True, inplace=True)
# replace multiple spaces in df column
df.replace(' +', ' ', regex=True, inplace=True)
# replace end punctuation in df column
df['Keyword'].replace('\s+$', '', regex=True, inplace=True)

df.fillna(0)
df.reset_index(drop=True, inplace=True)


In [6]:
df.head(10)

,Keyword,Word Before,Word After,Quote,Label,Lemma,Early Category Type,Early Word,Early Word Before,Early Word After,Early Quote,Lemma arabic
0,οὖλον,δὲ πολυφυὲς,· σάρκινα δὲ,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,Body Part,οὖλον,Body Part,οὖλον,δὲ πολυφυὲς,· σάρκινα δὲ,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,NaN
1,παρίσθμιον,τοῦ στόματος,", τὸ δὲ",Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,Body Part,παρίσθμιον,Body Part,παρίσθμιον,τοῦ στόματος,", τὸ δὲ",Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,NaN
2,πολυφυὲς,τὸ δὲ,οὖλον· σάρκινα,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,Adjectives/Qualities,πολυφυής,Adjectives/Qualities,πολυφυὲς,τὸ δὲ,οὖλον· σάρκινα,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,NaN
3,μόριον,δ’ ἄλλο,"σταφυλοφόρον, κίων","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",Body Part,μόριον,Body Part,μόριον,δ’ ἄλλο,"σταφυλοφόρον, κίων","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",NaN
4,ὀδόντες,Ἐντὸς δ’,ὀστέινοι. Εἴσω,Ἐντὸς δ’ ὀδόντες ὀστέινοι,Body Part,ὀδούς,Body Part,ὀδόντες,Ἐντὸς δ’,ὀστέινοι. Εἴσω,Ἐντὸς δ’ ὀδόντες ὀστέινοι,NaN
5,ὀστέινοι,δ’ ὀδόντες,. Εἴσω δ’,Ἐντὸς δ’ ὀδόντες ὀστέινοι,Adjectives/Qualities,ὀστέινος,Adjectives/Qualities,ὀστέινοι,δ’ ὀδόντες,. Εἴσω δ’,Ἐντὸς δ’ ὀδόντες ὀστέινοι,NaN
6,σταφυλοφόρον,ἄλλο μόριον,", κίων ἐπίφλεβος·","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",Adjectives/Qualities,σταφυλοφόρος,Adjectives/Qualities,σταφυλοφόρον,ἄλλο μόριον,", κίων ἐπίφλεβος·","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",NaN
7,ἐπίφλεβος,"σταφυλοφόρον, κίων",· ὃς ἐὰν,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",Adjectives/Qualities,ἐπίφλεβος,Adjectives/Qualities,ἐπίφλεβος,"σταφυλοφόρον, κίων",· ὃς ἐὰν,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",NaN
8,κίων,"μόριον σταφυλοφόρον,",ἐπίφλεβος· ὃς,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",Adjectives/Qualities,κίων,Adjectives/Qualities,κίων,"μόριον σταφυλοφόρον,",ἐπίφλεβος· ὃς,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",NaN
9,μέρος,Ἔτι προσώπου,τὸ μὲν,Ἔτι προσώπου μέρος τὸ μὲν ὂν τῷ πνευ...,Adjectives/Qualities,μέρος,Adjectives/Qualities,μέρος,Ἔτι προσώπου,τὸ μὲν,Ἔτι προσώπου μέρος τὸ μὲν ὂν τῷ πνευ...,NaN


In [7]:
# if any of the fields "KeyWord", "Quote", "Word Before", "Word After" are "0", drop the row
for w in ['Keyword', 'Quote', 'Word Before', 'Word After']:
    df = df[df[w] != 0]


In [165]:
# import requirements for converting the dataframe to Spacy Docs
from collections import defaultdict
from typing import List
from spacy.language import Language
from spacy.tokens import Doc, DocBin, Span
from spacy.util import filter_spans
from unicodedata import normalize
import regex
import random


# Convert the dataframe to Spacy Docs

In [227]:
d = {ord('\N{COMBINING ACUTE ACCENT}'):None, ord('\N{COMBINING COMMA ABOVE}'):None, ord('\N{COMBINING REVERSED COMMA ABOVE}'):None}

def find_word_index(sentence, word, word_before, word_after):
    """Find the index of a word in a sentence, it can appear multiple times but we return by word_before and word_after"""


    # find the index of the word in the sentence (return index of beginning of word)
    word_index = sentence.find(word)

    # return all chars of word before: [),.,·] (meaning none of these chars will be included) so we get only data contained within the sentence
    for char in [")", ".", "·"]:
        if word_before.find(char) != -1:
            word_before = word_before[word_before_blank.find(char)+1:]
        if word_after.find(char) != -1:
            word_after = word_after[:word_after_blank.find(char)]


    print ("cut_word_before_blank:", word_before_blank)
    print ("cut_word_after_blank:", word_after_blank)
    print ("sentence_blank:", sentence_blank)
    print ("word_blank:", word_blank)


    pattern = fr"{re.escape(word_before)}({(word_blank)}){re.escape(word_after)}"
    # add fuzzy matching with up to 3 mistakes
    pattern = fr"(?:{pattern}){{e<=3}}"


    print ("pattern:", pattern)
    print ("sentence_blank:", sentence_blank)

    match = regex.search(pattern, sentence)
    try:
        print ("match:", match.span(1))
        return match.span(1)
    except:

        return None



In [228]:
# normalize table
FORMAT = 'NFD'
for col in ['Keyword', 'Quote', 'Word Before', 'Word After']:
    df[col] = df[col].apply(lambda x: normalize(FORMAT, x))

In [260]:
# using word before and word after, we want to bring our data into the following format:
# TRAIN_DATA = [ (TEXT AS A STRING, {“entities”: [(START, END, LABEL)]}) ]

TRAIN_DATA = []
train_data_df = pd.DataFrame(columns=['text', 'keyword', 'index', 'label', 'lemma'])
for index, row in df.iterrows():

    train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
    print ("******")

cut_word_before_blank: δὲπολυφυὲς
cut_word_after_blank: ·σάρκιναδὲ
sentence_blank: Καὶτὸμὲνδιφυὲςτοῦστόματοςπαρίσθμιον,τὸδὲπολυφυὲςοὖλον
word_blank: οὖλον
pattern: (?:δὲ\ πολυφυὲς(οὖλον)){e<=3}
sentence_blank: Καὶτὸμὲνδιφυὲςτοῦστόματοςπαρίσθμιον,τὸδὲπολυφυὲςοὖλον
match: (68, 76)
******
cut_word_before_blank: τοῦστόματος
cut_word_after_blank: ,τὸδὲ
sentence_blank: Καὶτὸμὲνδιφυὲςτοῦστόματοςπαρίσθμιον,τὸδὲπολυφυὲςοὖλον
word_blank: παρίσθμιον
pattern: (?:τοῦ\ στόματος(παρίσθμιον),\ τὸ\ δὲ){e<=3}
sentence_blank: Καὶτὸμὲνδιφυὲςτοῦστόματοςπαρίσθμιον,τὸδὲπολυφυὲςοὖλον
match: (36, 48)
******
cut_word_before_blank: τὸδὲ
cut_word_after_blank: οὖλον·σάρκινα
sentence_blank: Καὶτὸμὲνδιφυὲςτοῦστόματοςπαρίσθμιον,τὸδὲπολυφυὲςοὖλον
word_blank: πολυφυὲς
pattern: (?:τὸ\ δὲ(πολυφυὲς)οὖλον){e<=3}
sentence_blank: Καὶτὸμὲνδιφυὲςτοῦστόματοςπαρίσθμιον,τὸδὲπολυφυὲςοὖλον
match: (58, 68)
******
cut_word_before_bl

In [261]:
# compare text[index] with keyword
test_data_df = pd.DataFrame(columns=['text', 'real_keyword', 'function_found'])
# count nulls in train_data_df
nulls = train_data_df.isnull().sum()
# drop nulls in train_data_df
train_data_df = train_data_df.dropna()
# iterate over all rows in train_data_df
for index, row in train_data_df.iterrows():
    test_data_df = test_data_df.append({'text': row['text'], 'real_keyword': row['keyword'], 'function_found': row['text'][row['index'][0]:row['index'][1]]}, ignore_index=True)


In [262]:
# print nulls in train_data_df
print ("Nulls", nulls)
# print shape of train_data_df
print ("\nShape", train_data_df.shape)


Nulls text        0
keyword     0
index      64
label       2
lemma       5
dtype: int64

Shape (1469, 5)


In [263]:
# sample test_data_df where real_keyword != function_found
test_data_df[test_data_df['real_keyword'] != test_data_df['function_found']].sample(10)

,text,real_keyword,function_found
1189,Τούτου δ' αἴτιον ὅτι ἐν μὲν τοῖς ἐξ α...,πάμμικρον,πάμμικρον
701,Αἱ μὲν γὰρ φέρουσιν εἰς τὰ πλάγια καὶ ...,εἰς,εἰς
1233,Πᾶσι δὲ τοῖς μὴ εὐθυεντέροις προϊοῦσιν...,ἐκ,ἐκ
1225,ἐνίοις μὲν γὰρ εὐρύτερον τὸ πρὸς τῇ ...,εὐρύτερον,εὐρύτερον
173,Ἔστι δ’ ἡ μὲν ἀρτηρία χονδρώδης τὴν φυ...,κατὰ,κατὰ
1424,αἵ τε γὰρ φλέβες εἰσὶ κοῖλαι καὶ αἵμ...,φλέβες,φλέβες
788,πλὴν ἐλάττους οἱ πόροι καὶ τὰ φλέβια π...,πόροι,πόροι
712,Πάλιν δ’ ἐντεῦθεν εἰς τέτταρας σχίζονται...,ἐπανακάμψασα,ἐπανακάμψασα
1310,Ὑπὸ δὲ τὸ ὑπόζωμα κεῖται ἡ κοιλία τοι...,εὐθὺς,εὐθὺς
502,ἐν ἑκάστῃ γὰρ ἔχει αἷμα τῶν κοιλιῶν...,ἐν,ἐν


# Create dictionaries from dendrosearch and conllu files (supplied by Jackobo)

In [167]:
PUNCTUATION = ['.', ")", ".", "·", "(", "[", "]", ":", ";", ",", "?", "!", "،", "_"]
# extract from df a dictionary {word: lemma}
lemma_dict = {}
for index, row in df.iterrows():
    lemma_dict[row['Keyword']] = row['Lemma']

# load dendrosearch lemma dictionary
dendrosearch_lemma_dict = {}
with open('../assets/dendrosearch_lemma_dict.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.split()
        # check if not punctuation
        if len(line) > 1 and line[0] not in PUNCTUATION:
            dendrosearch_lemma_dict[line[0]] = line[1]

# create dictionary from all conllu files
PATH = "../assets/Lemmatization_training_files/"
conllu_lemma_dict = {}

# iterate over all files in directory
for f in os.listdir(PATH):
    # if file is a conllu file
    with open(os.path.join(PATH, f), 'r', encoding='utf-8') as f:
        for line in f:
            # conll line is: id | keyword | lemma | pos | _
            # we want only keyword and lemma
            line = line.split()
            if len(line) > 2 and line[1] not in PUNCTUATION:
                conllu_lemma_dict[line[1]] = line[2]



[('ἦρξε', 'ἄρχω'),
 ('Πλειστόλας', 'Πλειστόλας'),
 ('πλίνθοις', 'πλίνθος'),
 ('ἀποβάλοιμι', 'ἀποβάλλω'),
 ('ἐνεδύετο', 'ἐνδύω'),
 ('εὐτυχέως', 'εὐτυχής'),
 ('Τέμπεα', 'Τέμπη'),
 ('ἠρώτα', 'ἐρωτάω'),
 ('στεινῷ', 'στενός'),
 ('φανοτάτου', 'φανός')]

## Create dictionary from INCEpTION files

In [266]:
from cassis import *
import zipfile
import tempfile

inception_dict = {}
inception_sentences = []

In [267]:
# extract all files in inception folder to temp folder
with tempfile.TemporaryDirectory() as tempdir:
    for f in os.listdir("../assets/NER_assets/INCEpTION_files/"):
        if f.endswith(".zip"):
            with zipfile.ZipFile(os.path.join("../assets/NER_assets/INCEpTION_files/", f), 'r') as zip_ref:
                zip_ref.extractall(tempdir)
    print (tempdir)
    # open typesystem and print content
    with open('{0}/{1}'.format(tempdir, "TypeSystem.xml"), 'rb') as f:
        typesystem = load_typesystem(f)

    # iterate over all files in temp folder
    for f in os.listdir(tempdir):
        # if file is a xmi file
        print (typesystem)
        if f.endswith(".xmi"):
            # load xmi file
            with open(os.path.join(tempdir, f), 'rb') as f:
                # load typesystem from temp folder
                cas = load_cas_from_xmi(f, typesystem=typesystem)
                for token in cas.select('de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Lemma'):
                    inception_dict[token.get_covered_text()] = token.value
                for sentence in cas.select("de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence"):
                    inception_sentences.append(sentence.get_covered_text())


C:\Users\roeyz\AppData\Local\Temp\tmpsiazm_an


In [200]:
# add to big dictionary and sample
big_dict = {**lemma_dict, **dendrosearch_lemma_dict, **conllu_lemma_dict, **inception_dict}

random.sample(list(big_dict.items()), 10)

[('ὁρμᾶτ’', 'ὁρμάω'),
 ('ὅλος', 'ὅλος'),
 ('Πύθιον', 'Πύθιος'),
 ('ἱδρώοντα', 'ἱδρόω'),
 ('συνεῖναί', 'σύνειμι'),
 ('σκληρύνητε', 'σκληρύνω'),
 ('καταλαμβανόμενον', 'καταλαμβάνω'),
 ('ἐκπεφευγέναι', 'ἐκφεύγω'),
 ('γινώσκομεν', 'γιγνώσκω'),
 ('κνώσσουσ’', 'κνώσσω')]

In [201]:
# create two copies of dictionary, one in NFD, other in NFKC
FIRST_FORMAT = 'NFD'
SECOND_FORMAT = 'NFKC'

big_dict_nfd = {}
big_dict_nfkc = {}

for key, value in big_dict.items():
    big_dict_nfd[normalize(FIRST_FORMAT, key)] = normalize(FIRST_FORMAT, value)
    big_dict_nfkc[normalize(SECOND_FORMAT, key)] = normalize(SECOND_FORMAT, value)

In [204]:
# check delta between two dictionaries
delta = {k: big_dict_nfd[k] for k in set(big_dict_nfd) - set(big_dict_nfkc)}
# delta size estimation
print ("delta size:", len(delta))

delta size: 168984


# Run NLP pipeline on INCEpTION and Coda files

In [269]:
# merge sentences from inception and coda

sentences = df['Quote'].tolist() + inception_sentences
sentences = list(set(sentences))
random.sample(sentences, 10)

['Ὀστέα χειρὸς εἰκοσιεπτὰ καὶ ποδὸς εἰκοσιτέσσαρα·',
 'κάτω δὲ πρὸς τὴν χεῖρα τὸ ὀστέον ἄρθρον ἔχει·',
 'Ταῦτα δὲ διαφύσεις ἔχει χονδρώδεις εἰς ὀξὺ συνηκούσας',
 'ὸ δὲ τὸ ὑπόζωμα κεῖται ἡ κοιλία τοῖς ζῴοις, τοῖς μὲν ἔχουσιν οἰσοφάγον ᾗ τελευτᾷ τοῦτο τὸ μόριον, τοῖς δὲ μὴ ἔχουσιν εὐθὺς πρὸς τῷ στόματι',
 "Ἀρτηρίη ἐξ ἑκατέρου φαρυγγέθρου τὴν ἔκφυσιν ποιευμένη ἐς \nἄκρον πνεύμονος τελευτᾷ, κρίκοις ξυγκειμένη ὁμορυσμοῖς, τῶν περιηγέων ἁπτομένη κατ' ἐπίπεδον ἀλλήλων.",
 'ὃς ἐὰν ἐξυγρανθεὶς φλεγμήνῃ, σταφυλὴ καλεῖται καὶ πνίγει',
 'Φυσωμένης δὲ τῆς ἀρτηρίας ἐν ἐνίοις μὲν οὐ κατάδηλον ποιεῖ, ἐν δὲ τοῖς μείζοσι τῶν ζῴων δῆλον ὅτι εἰσέρχεται τὸ πνεῦμα εἰς αὐτήν ',
 'εἶτα εἰς ἑκάτερον τὸ ἰσχίον ἀφανίζεται ἑκατέρα πρῶτον, ἔπειτα δῆλαι γίγνονται πάλιν διατεταμέναι πρὸς τὸ ἰσχίον',
 'Πρότερον δὲ τῇ θέσει ἡ ἀρτηρία κεῖται τοῦ οἰσοφάγου ἐν πᾶσι τοῖς ἔχουσιν αὐτήν·',
 'καὶ 

In [283]:
# run nlp pipeline on sentences
# install spacy grc model if not already installed
nlp = spacy.load("grc_proiel_sm") # I use small model for speed but you should use trf (transformer) model for better accuracy

# create DocBin object to store docs
db = DocBin()
# create df to record corrections
corrections_df = pd.DataFrame(columns=['sentence', 'token', 'lemma', 'lemma_corrected', 'dictionary'])
corrected_sentences = 0
for sentence in sentences:
    doc = nlp(sentence)
    # we use our dictionary to replace lemmas which the model didn't recognize correctly
    for token in doc:
        # if token text is a lexical value in our dictionary
        if token.text in big_dict_nfd:
            if token.lemma_ != big_dict_nfd[token.text]:
                corrections_df = corrections_df.append({'sentence': sentence,  'token': token.text, 'lemma': token.lemma_, 'lemma_corrected': big_dict_nfd[token.text], 'dictionary': 'big_dict_nfd'}, ignore_index=True)
                token.lemma_ = big_dict_nfd[token.text]

        elif token.text in big_dict_nfkc:
            if token.lemma_ != big_dict_nfkc[token.text]:
                corrections_df = corrections_df.append({'sentence': sentence,  'token': token.text, 'lemma': token.lemma_, 'lemma_corrected': big_dict_nfkc[token.text], 'dictionary': 'big_dict_nfkc'}, ignore_index=True)
                token.lemma_ = big_dict_nfkc[token.text]
    db.add(doc)


In [284]:
print ("corrections_df size:", corrections_df.shape)

corrections_df size: (4495, 5)


In [285]:
corrections_df.sample(10)


,sentence,token,lemma,lemma_corrected,dictionary
916,Πάλιν δ’ ἐντεῦθεν εἰς τέτταρας σχίζονται...,τοῦ,τοῦ,ὁ,big_dict_nfd
2985,Ἐξήρτηται δ' ἐκ τῆς μεγάλης φλεβὸςκαὶ ...,φλέβες,φλέβες,φλέψ,big_dict_nfd
1435,"Νεφροὶ δὲ ὁμοιορυσμοὶ, \nτὴν χροιὴν δὲ ἐναλίγκ...",δὲ,δέ,δὲ,big_dict_nfkc
1789,καὶ πάλιν ἕτε ραι διὰ τῶν βουβώνων καὶ...,ἡ,ἡ,ὁ,big_dict_nfd
1013,Ἡ μὲν οὖν ἀρτηρία τοῦτον ἔχει τὸν τρ...,ἐκβήξῃ,ἐκβήξῃ,ἐκβήσσω,big_dict_nfd
1653,Τὸ δ' ἐπίπλοον ἀπὸ μέσης τῆς κοιλίας ...,ἄλλοις,ἄλλοις,ἄλλος,big_dict_nfd
3448,προσάγεταί τε γὰρ καὶ λαμβάνει τούτῳ κα...,καὶ,καὶ,καί,big_dict_nfd
4187,πλὴν ἐλάττους οἱ πόροι καὶ τὰ φλέβια π...,ταύτης,ταύτης,οὗτος,big_dict_nfd
2546,διότι ἡ καρδίη αἰσθάνεταί τε μάλιστα κα...,αἱ,αἱ,ὁ,big_dict_nfd
1052,Πρότερον δὲ τῇ θέσει ἡ ἀρτηρία κεῖται...,Πρότερον,Πρότερος,πρότερος,big_dict_nfd


In [286]:
# find how many corrected by each dictionary
corrections_df.groupby('dictionary').count()


,sentence,token,lemma,lemma_corrected
dictionary,,,,
big_dict_nfd,4144,4144,4144,4144
big_dict_nfkc,351,351,351,351


In [287]:
corrections_df[corrections_df['dictionary'] == 'big_dict_nfkc'].sample(10)

,sentence,token,lemma,lemma_corrected,dictionary
129,Πρὸς δὲ τὸν καυλὸν τὸν τῆς κύστεως \r\nσυνήρ...,Πρὸς,Πρός,πρός,big_dict_nfkc
167,"ἐν τῇσιν ὀφρύσι διπλοῦν τὸ ὀστέον, καὶ ἡ σύγκλ...",σπονδύλους,σπονδύλος,σφόνδυλος,big_dict_nfkc
123,"Τελευταία γὰρ ἡ κύστις κεῖται, \r\nτὴν μὲν ἐξά...",ἰνώδεσιν,ἰνώδες,ἰνώδης,big_dict_nfkc
1990,ἀρτηρίαι δὲ ἐκ τούτου πεφύκασιν ἔνθεν \r\nκαὶ ...,δὲ,δέ,δὲ,big_dict_nfkc
210,Κοιλίη δὲ ἥπατι παρακειμένη κατ' εὐώνυμον μέρο...,ἥπατι,ἥπατι,ἧπαρ,big_dict_nfkc
3141,ἰσχύουσι δὲ μάλιστα ἐν ἐκείνοισι τοῦ σώματος κ...,δὲ,δέ,δὲ,big_dict_nfkc
1802,"δάκτυλοι ἄρθρα ἔχουσιν έκαστος τρία, ἂν μὲν ὑπ...",ἂν,ἐάν,ἄν,big_dict_nfkc
4023,τῷ δὲ \r\nδικραίῳ τούτῳ τὸ ὀστέον ἡ κνήμη καλε...,δὲ,δέ,δὲ,big_dict_nfkc
3078,ἄλλαι δύο φλέβες ἀπὸ τῆς κορυφῆς φέρονται ἐς τ...,κορυφῆς,κορυφή,κορύπτω,big_dict_nfkc
2601,"αἱ μὲν γὰρ σφίσιν \r\nἑωυτῇσι συμβάλλουσιν, αἱ...",ταύτῃ,οὗτος,ταύτῃ,big_dict_nfkc


In [288]:
corrections_df[corrections_df['dictionary'] == 'big_dict_nfd'].sample(10)

,sentence,token,lemma,lemma_corrected,dictionary
1000,Ἡ μὲν οὖν ἀρτηρία τοῦτον ἔχει τὸν τρ...,τὸ,τὸ,ὁ,big_dict_nfd
4315,Πᾶσαι δ’ αὗται εἰς μίαν φλέβα τελευτῶσι...,δ’,δ’,δέ,big_dict_nfd
3358,Ὑπὲρ δὲ τῶν ἐντέρων τὸ μεσεντέριόν ε...,μεσεντέριόν,μεσεντέριόν,μεσεντέριον,big_dict_nfd
967,πάντα γὰρ ὡς εἰπεῖν μεγάλα τὰ κερατοφό...,μεγάλα,μεγάλα,μέγας,big_dict_nfd
3515,μία δ’ ἑτέρα ἀφ’ ἑκατέρου τοῦ τόπου το...,μήνιγγα,μήνιγγα,μῆνιγξ,big_dict_nfd
2331,"τὸ μὲν γὰρ ἄνω καὶ κάτω, τὸ δ' ἔμπρο...",δ',δ',δέ,big_dict_nfd
2751,"τοῦτο γὰρ μεταξὺ τῆς τ' ἄνω, ἐν ᾗ το...",γὰρ,γὰρ,γάρ,big_dict_nfd
2866,Δύο φλέβες εἰσὶν ἐν τῷ θώρακι κατὰ τη...,τὸ,τὸ,ὁ,big_dict_nfd
3044,Διότι φημὶ τὸν ἐγκέφαλον εἶναι τὸν ἑρ...,φημὶ,φημὶ,φημί,big_dict_nfd
5,Ἔτι δ’ ἄλ λαι ἀπὸ τῆς μεγάλης φλεβὸς ...,ἀποσχίζονται,ἀποσχίζομαι,ἀποσχίζω,big_dict_nfd


In [ ]:
# save to spacy file
db.to_disk("../assets/NER_assets/corrected_sentences.spacy")
